# Whisper Fine-tuning

This is a heavily modified version of the Trelis Research notebook which allows for using a preprocessed version of the ATC0 dataset.

Purchase a copy of the original notebook at [Trelis.com](https://trelis.com/ADVANCED-transcription).

Notebook based off of the notebook built by Trelis Research. Find us at [Trelis.com](https://trelis.com), [YouTube](https://YouTube.com/@TrelisResearch) and on [HuggingFace](https://huggingface.co/Trelis).

*Subscribe to Trelis Research emails [here](https://trelis.substack.com) and get notified each time a new video tutorial is published.*

Built upon an [original notebook](https://colab.research.google.com/drive/1DOkD_5OUjFa0r5Ik3SgywJLJtEo2qLxO?usp=sharing#scrollTo=8d230e6d-624c-400a-bbf5-fa660881df25) by HuggingFace.

## Initial Setup

In [ ]:
!apt install ffmpeg -y

In [ ]:
!pip install ipywidgets
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install bitsandbytes datasets accelerate loralib
!pip install git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git@main

Link the notebook to the Hub is straightforward. Find your Hub authentication token [here](https://huggingface.co/settings/tokens):

In [ ]:
from huggingface_hub import login

login()

In [ ]:
# Select CUDA device index
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Input model
model_name_or_path = "openai/whisper-medium" #you can also choose a larger model (e.g. medium or large)
language = "English"
language_abbr = "en"
task = "transcribe"
dataset_name = "CEC595/atc"

# Output model path (for pushing to HuggingFace)
# org = "Trelis"
trained_adapter_name = "/kaggle/working/whisper-small-llm-atc-adapters"
trained_model_name = "/kaggle/working/whisper-small-llm-atc"

# trained_adapter_repo = org + "/" + trained_adapter_name
# trained_model_repo = org + '/' + trained_model_name

## Load Dataset

In [ ]:
from datasets import DatasetDict, load_dataset
import os

dataset = load_dataset('/kaggle/input/atc0-mini-preprocessed/pickled')

train = dataset['train']
validation = dataset['validation']

dataset = DatasetDict()

dataset["train"] = train
dataset["validation"] = validation

print(dataset)

## Prepare Feature Extractor, Tokenizer and Data

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path)

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path, language=language, task=task)

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(model_name_or_path, language=language, task=task)

### Prepare Data

In [ ]:
dataset["train"]

In [ ]:
dataset["validation"]

In [ ]:
print(dataset)

## Training and Evaluation

### Define a Data Collator

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Let's initialise the data collator we've just defined:

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

### Evaluation Metrics

We'll use the word error rate (WER) metric, the 'de-facto' metric for assessing
ASR systems. For more information, refer to the WER [docs](https://huggingface.co/metrics/wer). We'll load the WER metric from 🤗 Evaluate:

In [ ]:
import evaluate

metric = evaluate.load("wer")

We then simply have to define a function that takes our model
predictions and returns the WER metric. This function, called
`compute_metrics`, first replaces `-100` with the `pad_token_id`
in the `label_ids` (undoing the step we applied in the
data collator to ignore padded tokens correctly in the loss).
It then decodes the predicted and label ids to strings. Finally,
it computes the WER between the predictions and reference labels:

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

### Load a Pre-Trained Checkpoint

Now let's load the pre-trained Whisper `small` checkpoint. Again, this
is trivial through use of 🤗 Transformers!

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(model_name_or_path, load_in_8bit=False, device_map="auto")

# model.hf_device_map - this should be {" ": 0}

Override generation arguments - no tokens are forced as decoder outputs (see [`forced_decoder_ids`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.forced_decoder_ids)), no tokens are suppressed during generation (see [`suppress_tokens`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.suppress_tokens)):

In [ ]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

### Apply LoRA

Here comes the magic with `peft`! Let's load a `PeftModel` and specify that we are going to use low-rank adapters (LoRA) using `get_peft_model` utility function from `peft`.

In [ ]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

model = get_peft_model(model, config)

model.print_trainable_parameters()

We are ONLY using **~1%** of the total trainable parameters, thereby performing **Parameter-Efficient Fine-Tuning**

### Define the Training Configuration

In the final step, we define all the parameters related to training. For more detail on the training arguments, refer to the Seq2SeqTrainingArguments [docs](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments).

In [ ]:
from transformers import Seq2SeqTrainingArguments

output_dir = 'cec595-whisper'

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,  # change to a repo name of your choice
    per_device_train_batch_size=3, # increase to 16 for larger datasets
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-3,
    report_to="none",
    # warmup_steps=50,
    num_train_epochs=5,
    evaluation_strategy="epoch",
    fp16=True,
    per_device_eval_batch_size=1,
    generation_max_length=128,
    logging_steps=1,
    remove_unused_columns=False,  # required as the PeftModel forward doesn't have the signature of the wrapped model's forward
    label_names=["labels"],  # same reason as above
    predict_with_generate=True,
    save_steps=0.2, #if you wish to save checkpoints
)

**Important Notes:**
1. `remove_unused_columns=False` and `label_names=["labels"]` are required as the PeftModel's forward doesn't have the signature of the base model's forward.

2. If using INT8 - INT8 training required autocasting. `predict_with_generate` can't be passed to Trainer because it internally calls transformer's `generate` without autocasting leading to errors.

3. If using INT8 - Because of point 2, `compute_metrics` shouldn't be passed to `Seq2SeqTrainer` as seen below. (commented out)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [ ]:
trainer.train()

### Push adapters to hub and merge adapters onto the model

In [ ]:
adapter_to_push = f"{output_dir}/checkpoint-4770"

In [ ]:
from peft import PeftModel
model = WhisperForConditionalGeneration.from_pretrained(model_name_or_path, load_in_8bit=False, device_map="auto")

model = PeftModel.from_pretrained(
    model,
    adapter_to_push,
)

In [ ]:
model = model.merge_and_unload()

In [ ]:
print(model)

## Evaluation

In [ ]:
model.save_pretrained(trained_model_name)

In [ ]:
processor.save_pretrained(trained_model_name)

In [ ]:
tokenizer.save_pretrained(trained_model_name)

In [ ]:
# whisper_finetuned_asr = pipeline(
#     "automatic-speech-recognition",
#     model=trained_model_name,
#     chunk_length_s=30, #this splits the input into 30 second chunks, the max length for whisper.
#     # generate_kwargs={"forced_decoder_ids": forced_decoder_ids},
#     device="cuda" if torch.cuda.is_available() else "cpu" #ensures that the GPU is used, for speed up.
# )

## Push Model and Processor to Hub

In [ ]:
# ## Run this code if you face utf locale errors
# # https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
# import locale
# def getpreferredencoding(do_setlocale = True):
#     return "UTF-8"
# locale.getpreferredencoding = getpreferredencoding